In [2]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

### 2016-01 Recuperatorio

Una red social almacena el contenido de los chats entre sus usuarios
en un RDD que tiene registros con el siguiente formato: (chat_id,
user_id, nickname, text). Queremos saber cuál es el usuario (user_id)
que mas preguntas hace en sus chats, contabilizamos una pregunta por
cada caracter “?” que aparezca en el campo text. Programar en Spark
un programa que identifique al usuario preguntón.

In [2]:
chats = [
    (1, 1, 'damu', 'Qué es esto?'),
    (2, 2, 'martin', 'Un chat!'),
    (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
    (4, 2, 'martin', 'Sí! Cómo sabias?'),
    (5, 1, 'damu', 'Adivine'),
    (6, 3, 'luis', 'Hola!')
]

In [3]:
data = sc.parallelize(chats);

In [4]:
data.collect()

[(1, 1, 'damu', 'Qué es esto?'),
 (2, 2, 'martin', 'Un chat!'),
 (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
 (4, 2, 'martin', 'Sí! Cómo sabias?'),
 (5, 1, 'damu', 'Adivine'),
 (6, 3, 'luis', 'Hola!')]

In [5]:
data.map(lambda x: (x[1], x[3].count('?')))\
    .reduceByKey(lambda x, y: x+y)\
    .reduce(lambda x, y: x if (x[1] > y[1]) else y)

(1, 3)

### 2016-01 Parcial

UBER almacena en un cluster todos los datos sobre el movimiento y
viajes de todos sus vehículos. Existe un proceso que nos devuelve un
RDD llamado trip_summary con los siguientes campos: (driver_id,
car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled),
Programar usando Py Spark un programa que nos indique cual fue el
conductor con mayor promedio de distancia recorrida por viaje para
Abril de 2016.

In [11]:
trips = [
    (1, 1, 1, 1, '20160101', 10),
    (2, 2, 2, 2, '20160202', 20),
    (1, 1, 3, 1, '20160402', 15),
    (1, 1, 4, 3, '20160405', 20),
    (2, 2, 5, 4, '20160410', 25),
    (3, 3, 6, 3, '20160415', 15),
    (2, 2, 7, 1, '20160420', 40),
    (3, 3, 8, 2, '20160505', 80)
]

In [12]:
data = sc.parallelize(trips);

In [13]:
data.collect()

[(1, 1, 1, 1, '20160101', 10),
 (2, 2, 2, 2, '20160202', 20),
 (1, 1, 3, 1, '20160402', 15),
 (1, 1, 4, 3, '20160405', 20),
 (2, 2, 5, 4, '20160410', 25),
 (3, 3, 6, 3, '20160415', 15),
 (2, 2, 7, 1, '20160420', 40),
 (3, 3, 8, 2, '20160505', 80)]

In [26]:
data.filter(lambda x: (x[4] > '20160400') and (x[4] < '20160500'))\
    .map(lambda x: (x[0], (1, x[5])))\
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
    .map(lambda x: (x[0], x[1][1]/x[1][0]))\
    .reduce(lambda x, y: x if x[1] > y[1] else y)
    
# El último map no es necesario, agregado solo por claridad. 
# Sin el map lo único que cambia es la comparación que se hace en el reduce.

(2, 32)

### 2015-02 2do Recuperatorio

Un telescopio registra automaticamente la luminosidad de distintas
estrellas generando un RDD con registros de tipo (star_id,
magnitude,spectral_type, timestamp). Queremos obtener un listado de
estrellas que tienen el mismo tipo espectral e igual promedio de
magnitud una vez redondeado el mismo a un decimal. El resultado
debe ser una lista en donde cada elemento de la lista es una lista de ids
de estrellas similares.

In [8]:
stars = [
    (1, 5, 1, 1),
    (2, 10, 1, 1),
    (3, 6, 1, 1),
    (4, 5.5, 2, 1),
    (1, 6, 1, 2),
    (2, 9, 1, 2),
    (3, 5, 1, 2),
    (1, 5.5, 1, 3),
    (2, 11, 1, 3),
    (3, 5.5, 1, 3)
]

In [9]:
data = sc.parallelize(stars);

In [10]:
data.collect()

[(1, 5, 1, 1),
 (2, 10, 1, 1),
 (3, 6, 1, 1),
 (4, 5.5, 2, 1),
 (1, 6, 1, 2),
 (2, 9, 1, 2),
 (3, 5, 1, 2),
 (1, 5.5, 1, 3),
 (2, 11, 1, 3),
 (3, 5.5, 1, 3)]

In [14]:
data.map(lambda x: (x[0], (x[2], x[1], 1)))\
    .reduceByKey(lambda x, y: (x[0], x[1]+y[1], x[2]+y[2]))\
    .map(lambda x: ((x[1][0], x[1][1]/x[1][2]), x[0]))\
    .groupByKey()\
    .map(lambda x: (x[0], list(x[1])))\
    .collect()
    
# El último map es simplemente para convertir la sequencia de spark en una lista de python para poder verla

[((1, 5.5), [1, 3]), ((1, 10.0), [2]), ((2, 5.5), [4])]

### 2015-01 2do Recuperatorio

Tenemos una colección de documentos (textos) almacenados en un cluster. Se quiere establecer un ranking de los patrones mas frecuentes para la aparición de letras en las palabras. Por ejemplo “sister”, “ejects” , “ninety” y “amazon” responden al patrón “abacde”. Programar en map-reduce un programa que genere como resultado un
listado de tipo (patron, frecuencia) indicando cuántas veces aparece cada patrón en la colección de documentos.

In [17]:
textFile = sc.textFile("README.md")
words = textFile.flatMap(lambda line: line.split())
words.collect()

['#',
 'README',
 '#',
 'Repositorio',
 'creado',
 'para',
 'ir',
 'subiendo',
 'los',
 'diferentes',
 'algoritmos',
 'que',
 'vayamos',
 'programando',
 'y',
 'para',
 'la',
 'documentacion',
 'que',
 'vayamos',
 'recolectando',
 'a',
 'medida',
 'que',
 'avanzamos',
 'con',
 'el',
 'tp',
 '###',
 'Estructura',
 'de',
 'directorios',
 '###',
 '.',
 '+--',
 '_data',
 '+--',
 '_docs',
 '+--',
 '_src',
 '|',
 '+--',
 'python',
 '|',
 '+--',
 'notebooks',
 'En',
 'data',
 'guardaremos',
 'los',
 'archivos',
 'train.csv',
 'y',
 'test.csv',
 'en',
 'nuestro',
 'local.',
 'En',
 'docs',
 'todo',
 'lo',
 'referente',
 'a',
 'diseño,',
 'informe,',
 'graficos,',
 'etc.',
 'En',
 'src',
 'vamos',
 'ir',
 'colocando',
 'codigo',
 'que',
 'nos',
 'ayude',
 'a',
 'resolver',
 'el',
 'TP.',
 'Supuse',
 'clasificarlo',
 'por',
 'lenguajes',
 'por',
 'si',
 'tenemos',
 'que',
 'utilizar',
 'varios.',
 'En',
 'la',
 'carpeta',
 'notebooks',
 'se',
 'tiene',
 'que',
 'crear',
 '(si',
 'no',
 'esta',
 

In [64]:
def pattern(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    pat = ''
    found = ''
    for letter in word:
        if letter in found:
            pat += pat[found.index(letter)]
        else:
            found += letter
            if len(found) > len(letters):
                pat += '?'
            else:
                pat += letters[len(found)-1]
    return pat
   
words.map(lambda x: (pattern(x), 1))\
     .reduceByKey(lambda x, y: x+y)\
     .takeOrdered(10, lambda x: -x[1])

[('abc', 85),
 ('ab', 58),
 ('abcd', 39),
 ('abcde', 37),
 ('abcdef', 16),
 ('abcdefg', 16),
 ('a', 12),
 ('aa', 8),
 ('abcdefgh', 8),
 ('abcdde', 6)]

### 2015-2C Parcial

 Se tiene un RDD con las coordenadas de rectángulos de la forma
(x1,x2,y1,y2). Se pide programar en PySpark un programa que
encuentre el rectángulo de superficie mínima que contiene al punto
(w,z) 

In [14]:
rectangulos = [
    (1.2, 3.1, 0.9, 3),
    (1.2, -1.2, 5, 8),
    (-4, -1.2, 0.8, 3.7),
    (-4, -1.2, -3, -9.7),
    (4, 1.2, -3, 3.7),
    (1.2, 2.1, 0.9, 3)
]
point = (2,1)

def isInsideOfRectangle(rectangle,point):
    isInside = False
    if(rectangle[0] > rectangle[1]):
        isInside = (point[0]<=rectangle[0] and point[0]>=rectangle[1])
    else:
        isInside = (point[0]<=rectangle[1] and point[0]>=rectangle[0])
        
    if(rectangle[2] > rectangle[3]):
        isInside = (point[1]<=rectangle[2] and point[1]>=rectangle[3])
    else:
        isInside = (point[1]<=rectangle[3] and point[1]>=rectangle[2])
        
    return isInside

def surface(rectangle):
    x = abs(rectangle[0] - rectangle[1])
    y = abs(rectangle[2] - rectangle[3])
    return x*y

In [15]:
rdd = sc.parallelize(rectangulos)

rdd.filter(lambda x: isInsideOfRectangle(x,point))\
    .reduce(lambda x,y: x if surface(x)<surface(y) else y)

(1.2, 2.1, 0.9, 3)

### 2016-2C 1er Recuperatorio

Se tiene un RDD con información meteorológica de la forma (STATION_ID, YEAR, MONTH, DAY, HOUR, MINUTE, TEMPERATURE). Calcule primero la temperatura promedio para cada estación y día del año. Luego queremos encontrar cuál es la estación que ha sufrido una mayor variación absoluta de temperatura promedio entre un día y otro y cuál fue esa variación. Por ejemplo station_id = 43, variación = 43 grados. No es necesario indicar en qué fecha. Esto quiere decir que entre un día y otro el promedio de temperatura cambió en 18 grados (hacia arriba o abajo) para la estación 43.

In [81]:
import datetime
from dateutil.relativedelta import relativedelta

data = [
    (1,2016,9,19,15,30,16),
    (1,2016,9,19,16,30,15),
    (1,2016,9,20,16,30,15),
    (1,2016,9,21,16,30,13),
    (1,2016,9,21,12,30,12),
    (2,2016,9,19,15,30,16),
    (2,2016,9,19,16,30,15),
    (2,2016,9,20,16,30,15),
    (2,2016,9,21,16,30,13),
    (2,2016,9,21,12,30,12),
    (4,2016,9,21,16,30,13),
    (4,2016,9,21,12,30,12),
]

def diferenceDays(x,y):
    a = str(x[0])+'-'+str(x[1])+'-'+str(x[2])
    b = str(y[0])+'-'+str(y[1])+'-'+str(y[2])
    
    start = datetime.datetime.strptime(a, '%Y-%m-%d')
    ends = datetime.datetime.strptime(b, '%Y-%m-%d')
    
    diff = relativedelta(start, ends)
    
    return diff.days
    
def getMaxVariationOfTemperature(arr):
    prev = arr[0]
    maxVariation = 0
    for x in arr:
        if(diferenceDays(prev,x)):
            variation = abs(prev[3]-x[3])
            if(variation>maxVariation):
                maxVariation = variation
    return maxVariation

In [82]:
rdd = sc.parallelize(data)

rdd = rdd.map(lambda x: ((x[0],x[1],x[2],x[3]),(x[6],1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .map(lambda x: (x[0][0],(x[0][1],x[0][2],x[0][3],x[1][0]/x[1][1])))
    
rdd.collect()

[(1, (2016, 9, 20, 15.0)),
 (2, (2016, 9, 21, 12.5)),
 (1, (2016, 9, 19, 15.5)),
 (2, (2016, 9, 19, 15.5)),
 (4, (2016, 9, 21, 12.5)),
 (2, (2016, 9, 20, 15.0)),
 (1, (2016, 9, 21, 12.5))]

In [85]:
rdd.sortBy(lambda x: (x[1][1],x[1][2],x[1][3]))\
    .groupByKey().map(lambda x: (x[0],list(x[1])))\
    .map(lambda x: (x[0],getMaxVariationOfTemperature(x[1])))\
    .reduce(lambda x,y: x if x[1]>y[1] else y)
    #.reduceByKey(lambda x,y: abs(x[1][3]-y[1][3]) if ((y[1][2]-x[1][2]) == 1) else 0).take(1)\
    #.reduce(lambda x,y: x if x[1]>y[1] else y)

(2, 3.0)